In [8]:
import tkinter as tk
import re

import ipynb_importer
from publish_subscribe import Reader

In [1]:
class MainPanel:
    def __init__(self, reader, publisher):
        self.reader = reader
        self.publisher = publisher
        self.all_authors = ['李白', '杜甫', '王維', '白居易']
        self.cur_author = ""
    
    def start(self):
        """
        显示主界面
        """
        self.root = tk.Tk()
        self.root.geometry('400x400')
        self.root.title("古诗订阅")
        #self.root.iconbitmap('cat.ico')
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.root.mainloop()
        
    def add_subscription(self):
        """
        用户点击"新增订阅"按钮，显示该界面
        """
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        #self.add_panel.iconbitmap('cat.ico')
        present_author_label = tk.Label(self.add_panel, text=f"已订阅作者：{self.reader.authors}", font=(None,10)).pack(pady=20)
        label = tk.Label(self.add_panel, text="新增订阅:", font=(None, 10)).pack(pady=20)
        self.new_author_entry = tk.Entry(self.add_panel)
        self.new_author_entry.pack()
        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10), command=self.check_new_author).pack(pady=20)
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()
    
    def check_new_author(self):
        """
        用户点击"确认"按钮后，检查输入是否合法
        
        """
        new_author = self.new_author_entry.get()
        if new_author not in self.all_authors:
            self.hint_label.config(text=f"目前仅支持订阅:{self.all_authors}")
        elif new_author in self.reader.authors:
            self.hint_label.config(text="该作者已订阅，请重新输入")
        else:
            tmp_tk = tk.Tk()
            tmp_tk.geometry("300x200")
            tk.Label(tmp_tk, text="订阅成功", font=(None, 20)).pack(pady=20)
            tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)
            self.reader.subscribeToPublisher(new_author, self.publisher)
            self.add_panel.destroy()
            self.update_main_panel()
            
    def update_main_panel(self):
        """
        用户新增订阅后，更新主界面
        """
        for widget in self.root.winfo_children():
            widget.destroy()
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        for author in self.reader.authors:
            #修改了此处，让author作为参数传入被调用函数
            button = tk.Button(self.root, text=author, width=13, font=(None,12),command=lambda: self.show_titles(author))
            button.pack(pady=10)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.root.update()
        
    def show_titles(self, author):
        """
        用户点击主界面的“作者名”按钮：
        
        1. 显示该目前接收到的该作者的全部诗的标题，要求滚动条显示（tkinter的Scrollbar控件）
        
        2. 点击标题，显示诗的具体内容（提示：可以使用 Listbox 控件的bind方法，动作为 <ListboxSelect>)
        
        3. 选做：增加诗题检索功能，获取标题符合条件的诗
        
        
        Notice：
        
        1. Demo 仅为参照，布局可按自己的喜好设计
        
        2. 选做题的检索方式自定，全字匹配、正则检索等都可以
        
        3. 选做题不强制，占 1-2 分，不想做检索也可设计其他功能
        
        """
        # TODO
        self.poet=tk.Tk()
        self.poet.geometry('300x200')
        
        fml=tk.Frame(self.poet)
        fml.pack()
        self.poem_name_entry=tk.Entry(fml)
        self.poem_name_entry.pack(side=tk.LEFT,pady=20)
        search=tk.Button(fml,text="search",width=5, font=(None,10),command=lambda: self.search(author))
        ret=tk.Button(fml,text="back",width=5, font=(None,10),command=lambda: self.back(author))
        ret.pack(side=tk.RIGHT)
        search.pack(side=tk.RIGHT)
        
        scrollbar=tk.Scrollbar(self.poet)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.theLB=tk.Listbox(self.poet,yscrollcommand=scrollbar.set)
        
        self.tmp_author=author
        poem_list=self.reader.poems[author]
        for i in poem_list:
            self.theLB.insert(tk.END,i[0])
        
        self.theLB.bind("<<ListboxSelect>>",self.show_poem)
        self.theLB.pack(fill=tk.BOTH)
        scrollbar.config(command=self.theLB.yview)
        self.theLB.mainloop()
                 
    def show_poem(self,event):
        idx=self.theLB.curselection()
        index=int(idx[0])
        poem=self.reader.poems[self.tmp_author][index]
        tmp_tk = tk.Tk()
        tmp_tk.geometry("300x200")
        note=""
        for i in poem[2:]:
            note+=i+"\n"
        msg=tk.Message(tmp_tk,text=note)
        msg.config()
        msg.pack()
        
    def back(self,author):
        poem_list=self.reader.poems[author]
        self.theLB.delete(0,tk.END)
        for i in poem_list:
            self.theLB.insert(tk.END,i[0])
        self.poet.update()

    def search(self,author):
        poem_list=self.reader.poems[author]
        self.theLB.delete(0,tk.END)
        target=self.poem_name_entry.get()
        length=len(target)
        for i in poem_list:
            title=i[0]
            if(target==title[0:length]):
                self.theLB.insert(tk.END,i[0])
        self.poet.update()
            